In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import cv2
from PIL import Image
import tensorflow as tf
from tensorflow import keras

In [ ]:
img_path='/kaggle/input/cardiac-catheterization/Cardiac_Catheterization/train/images/0655[0]_47.png'
img_pillow=Image.open(img_path)
img_color=img_pillow.convert('RGB')
img_arr=np.array(img_color)
plt.imshow(img_arr)

In [ ]:
img_mask='/kaggle/input/cardiac-catheterization/Cardiac_Catheterization/train/masks/0655[0]_47.png'
img_pillow=Image.open(img_mask)
img_color=img_pillow.convert('RGB')
img_arr=np.array(img_color)
plt.imshow(img_arr)

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split
from keras.layers import Dense, Conv2D, MaxPool2D , Flatten,LeakyReLU,BatchNormalization
import tensorflow as tf
from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers import Conv2D, Conv2DTranspose,DepthwiseConv2D
from keras.layers import MaxPooling2D, GlobalMaxPool2D, Concatenate
from keras.layers import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam,SGD
from keras.preprocessing.image import array_to_img, img_to_array, load_img
from tqdm import tqdm

In [ ]:
im_width = 256
im_height = 256
ids1 = next(os.walk("/kaggle/input/cardiac-catheterization/Cardiac_Catheterization/train/images/"))[2]
X = np.zeros((len(ids1), im_height, im_width, 1), dtype=np.float32)
y = np.zeros((len(ids1), im_height, im_width, 1), dtype=np.float32)
# tqdm is used to display the progress bar
for n, id_ in tqdm(enumerate(ids1), total=len(ids1)):
    # Load images
    img = load_img("/kaggle/input/cardiac-catheterization/Cardiac_Catheterization/train/images/" + id_, color_mode='grayscale')
    x_img = img_to_array(img)
    x_img = resize(x_img, (256,256, 1), mode='constant', preserve_range=True)
    # Load masks
    mask = img_to_array(load_img("/kaggle/input/cardiac-catheterization/Cardiac_Catheterization/train/masks/" + id_, color_mode='grayscale'))
    mask = resize(mask, (256,256, 1), mode='constant', preserve_range=True)
    # Save images
    X[n] = x_img / 255.0
    y[n] = mask / 255.0

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3,shuffle=True)

In [ ]:
X_train.shape

In [ ]:
def conv_block(inputs, num_filters):
    x = Conv2D(num_filters, 3, padding="same")(inputs)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    x = Conv2D(num_filters, 3, padding="same")(x)
    x = BatchNormalization()(x)
    x = Activation("relu")(x)

    return x
def encoder_block(inputs, num_filters):
    x = conv_block(inputs, num_filters)
    p = MaxPool2D((2, 2))(x)
    return x, p
def decoder_block(inputs, skip_features, num_filters):
    x = Conv2DTranspose(num_filters, 2, strides=2, padding="same")(inputs)
    x = Concatenate()([x, skip_features])
    x = conv_block(x, num_filters)
    return x

In [ ]:
def unet(input_shape):
    inputs = Input(input_shape)

    s1, p1 = encoder_block(inputs, 64)
    s2, p2 = encoder_block(p1, 128)
    s3, p3 = encoder_block(p2, 256)
    s4, p4 = encoder_block(p3, 512)

    b1 = conv_block(p4, 1024)

    d1 = decoder_block(b1, s4, 512)
    d2 = decoder_block(d1, s3, 256)
    d3 = decoder_block(d2, s2, 128)
    d4 = decoder_block(d3, s1, 64)

    outputs = Conv2D(1, 1, padding="same", activation="sigmoid")(d4)

    model = Model(inputs, outputs, name="UNET")
    return model

model = unet((256,256,1))
model.summary()

In [ ]:
!pip install --upgrade tensorflow

In [ ]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

history = model.fit(X_train, y_train, batch_size=4, epochs=3)

In [ ]:
model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=['accuracy'])

model.fit(X_train, y_train, batch_size=32, epochs=3)

In [ ]:
model.save('/kaggle/working/Cardiac_Catheterization_Model.hdf5')

In [ ]:
loss, accuracy = model.evaluate(X_test, y_test)
print("Test Accuracy:", accuracy)

y_pred = model.predict(X_test)

y_pred_binary = (y_pred > 0.5).astype(np.uint8)
y_test = (y_test > 0.5).astype(np.uint8)

In [ ]:
model.save('/kaggle/working/CC_Model.hdf5')

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from skimage.transform import resize
import numpy as np

# Assuming 'im_width' and 'im_height' are defined earlier
im_width = 256
im_height = 256

# Load and preprocess a single image
img_path = '/kaggle/input/cardiac-catheterization/Cardiac_Catheterization/train/images/0655[0]_54.png'
img = load_img(img_path, color_mode='grayscale')  # Use 'color_mode' instead of 'grayscale'
x_img = img_to_array(img)
x_img = resize(x_img, (im_height, im_width), mode='constant', preserve_range=True)  # Remove the last dimension
x_img /= 255.0

# Expand dimensions to match model input shape
x_img = np.expand_dims(x_img, axis=0)

# Perform prediction
predictions = model.predict(x_img)
prediction = predictions.squeeze()  # Remove the batch dimension
plt.imshow(prediction, cmap='gray')  # Display as grayscale
plt.show()

In [ ]:
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt

# Threshold the prediction to obtain a binary mask
threshold = 0.5  # Adjust threshold as needed
binary_mask = prediction > threshold

# Perform connected component analysis
labeled_mask = label(binary_mask)

# Get region properties
regions = regionprops(labeled_mask)

# Draw bounding boxes around regions
fig, ax = plt.subplots()
ax.imshow(x_img.squeeze(), cmap='gray')

for region in regions:
    # Get bounding box coordinates
    minr, minc, maxr, maxc = region.bbox
    width = maxc - minc
    height = maxr - minr
    area = region.area
    
    # Draw bounding box
    rect = plt.Rectangle((minc, minr), width, height, fill=False, edgecolor='white', linewidth=2)
    ax.add_patch(rect)

plt.show()

In [ ]:
from keras.preprocessing.image import load_img, img_to_array
from skimage.transform import resize
import numpy as np
from skimage.measure import label, regionprops
import matplotlib.pyplot as plt

# Assuming 'im_width' and 'im_height' are defined earlier
im_width = 256
im_height = 256

# Load and preprocess a single image
img_path = '/kaggle/input/cardiac-catheterization/Cardiac_Catheterization/train/images/0655[0]_54.png'
img = load_img(img_path, color_mode='grayscale')  # Use 'color_mode' instead of 'grayscale'
x_img = img_to_array(img)
x_img = resize(x_img, (im_height, im_width), mode='constant', preserve_range=True)  # Remove the last dimension
x_img /= 255.0

# Expand dimensions to match model input shape
x_img = np.expand_dims(x_img, axis=0)

# Perform prediction
predictions = model.predict(x_img)
prediction = predictions.squeeze()  # Remove the batch dimension

# Threshold the prediction to obtain a binary mask
threshold = 0.5  # Adjust threshold as needed
binary_mask = prediction > threshold

# Perform connected component analysis
labeled_mask = label(binary_mask)

# Get region properties
regions = regionprops(labeled_mask)

# Define margin size (in pixels)
margin = 5  # Adjust margin size as needed

# Display the original image with bounding boxes around regions
fig, ax = plt.subplots()
ax.imshow(x_img.squeeze(), cmap='gray')

for region in regions:
    # Get bounding box coordinates
    minr, minc, maxr, maxc = region.bbox
    width = maxc - minc
    height = maxr - minr
    area = region.area
    
    # Expand bounding box with margin
    minr -= margin
    minc -= margin
    maxr += margin
    maxc += margin
    width = maxc - minc
    height = maxr - minr
    
    # Draw bounding box with margin
    rect = plt.Rectangle((minc, minr), width, height, fill=False, edgecolor='green', linewidth=2)
    ax.add_patch(rect)

    # Print region properties
    print(f"Height: {height}, Width: {width}, Area: {area}")

plt.show()